<a href="https://colab.research.google.com/github/tmyc131/lrc_file_crawler/blob/master/kanana_1_5_2_1b_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

Model page: https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="kakaocorp/kanana-1.5-2.1b-base")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")

In [ ]:
# requirements.txt
transformers>=4.30.0
torch>=2.0.0
pandas>=1.5.0
openpyxl>=3.1.0
python-docx>=0.8.11
PyQt5>=5.15.0
scikit-learn>=1.3.0
numpy>=1.24.0
sentence-transformers>=2.2.0


In [2]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.4 MB/s eta 0:00:00


In [3]:
# 기본 라이브러리 임포트
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pickle
from pathlib import Path

# AI 모델 관련
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 문서 처리
from docx import Document
from docx.shared import Inches
import openpyxl

# GUI
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *


ModuleNotFoundError: No module named 'PyQt5'